In [1]:
using Pkg
Pkg.DEFAULT_IO[] = stdout

IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(42) open, 0 bytes waiting)))

In [2]:
Pkg.activate("/Users/zyxu/Documents/julia/demo")

  Activating environment at `~/Documents/julia/demo/Project.toml`


In [3]:
Pkg.status()

     Project demo v0.1.0
      Status `~/Documents/julia/demo/Project.toml`
  [336ed68f] CSV v0.9.6
  [324d7699] CategoricalArrays v0.10.2
  [7876af07] Example v0.5.4 `~/.julia/dev/Example`
  [ced58d7a] HighDimMixedModels v0.1.0 `~/.julia/dev/HighDimMixedModels`
  [a1dec852] Metida v0.12.4
  [ff71e718] MixedModels v4.6.0
  [14b8a8f1] PkgTemplates v0.7.19
  [bd369af6] Tables v1.6.0
  [44d3d7a6] Weave v0.10.10


In [4]:
using DataFrames
using LinearAlgebra
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels
using Metida
using CSV
using CategoricalArrays;

In [5]:
using HighDimMixedModels

In [9]:
#df = CSV.File(joinpath(dirname(pathof(Metida)), "..", "test", "csv", "1fptime.csv"); types = [String, Float64, String, String]) |> DataFrame
df = CSV.File(joinpath(dirname(pathof(Metida)), "..", "test", "csv", "1freparma.csv"); types = [String, Float64, String, String]) |> DataFrame

df[!,:response] = map(x->parse(Float64,x),df[:,:response])
df[!,:subject] = map(x->parse(Float64,x),df[:,:subject])
df[!,:time] = map(x->parse(Float64,x),df[:,:time])
transform!(df, :factor => categorical, renamecols=false)
first(df,5)

,subject,factor,response,time
,Float64,Cat…,Float64,Float64
1,1.0,1.0,46.951,1.0
2,1.0,1.0,46.3568,2.0
3,1.0,1.0,49.1365,3.0
4,1.0,1.0,48.1,4.0
5,1.0,1.0,48.2826,5.0


In [7]:
lmm = LMM(@formula(response ~ 0 + subject + time), df;
random = VarEffect(@covstr(1|factor), SI)
)
fit!(lmm)

Linear Mixed Model: response ~ 0 + subject + time
Random 1: 
    Model: 1|factor
    Type: SI (1), Subjects: 2
Repeated: 
    Residual only
Blocks: 2, Maximum block size: 120
Status: converged (No Errors)
    -2 logREML: 1346.31    BIC: 1357.25

    Fixed-effects parameters:
──────────────────────────────────────────────
             Coef.  Std. Error     z  Pr(>|z|)
──────────────────────────────────────────────
subject  0.0356667   0.0710864  0.50    0.6159
time     0.648923    0.085435   7.60    <1e-13
──────────────────────────────────────────────
    Variance components:
    θ vector: [47.0302, 3.802]
  Random 1   σ²    var   2211.84
  Residual   σ²    var   14.4552


In [8]:
fm = @formula(response ~ 0 + subject + time + (1|factor))
mm = fit(MixedModel, fm, df, REML=true)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /Users/zyxu/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
Minimizing 66 	 Time: 0:00:00 ( 6.15 ms/it)


|             |   Est. |     SE |    z |      p | σ_factor |
|:----------- | ------:| ------:| ----:| ------:| --------:|
| subject     | 0.0357 | 0.0711 | 0.50 | 0.6159 |          |
| time        | 0.6489 | 0.0854 | 7.60 | <1e-13 |          |
| (Intercept) |        |        |      |        |  47.0302 |
| Residual    | 3.8020 |        |      |        |          |


In [9]:
show(mm)

Linear mixed model fit by REML
 response ~ 0 + subject + time + (1 | factor)
 REML criterion at convergence: 1346.311176818838

Variance components:
            Column    Variance Std.Dev.
factor   (Intercept)  2211.8406 47.0302
Residual                14.4552  3.8020
 Number of obs: 240; levels of grouping factors: 2

  Fixed-effects parameters:
──────────────────────────────────────────────
             Coef.  Std. Error     z  Pr(>|z|)
──────────────────────────────────────────────
subject  0.0356667   0.0710864  0.50    0.6159
time     0.648923    0.085435   7.60    <1e-13
──────────────────────────────────────────────

In [14]:
mm.optsum

|                          |                    |
|:------------------------ |:------------------ |
| **Initialization**       |                    |
| Initial parameter vector | [1.0]              |
| Initial objective value  | 1523.5690721546475 |
| **Optimizer settings**   |                    |
| Optimizer (from NLopt)   | `LN_BOBYQA`        |
| Lower bounds             | [0.0]              |
| `ftol_rel`               | 1.0e-12            |
| `ftol_abs`               | 1.0e-8             |
| `xtol_rel`               | 0.0                |
| `xtol_abs`               | [1.0e-10]          |
| `initial_step`           | [0.75]             |
| `maxfeval`               | -1                 |
| `maxtime`                | -1.0               |
| **Result**               |                    |
| Function evaluations     | 65                 |
| Final parameter vector   | [12.3699]          |
| Final objective value    | 1346.3112          |
| Return code              | `FTOL_REACHED`     |


### High Dim Mixed Models

In [10]:
fm = @formula(response ~ 0 + subject + time + (1|factor))
HMM = highDimMixedModel(fm, df, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ HighDimMixedModels /Users/zyxu/.julia/dev/HighDimMixedModels/src/bricks.jl:12


In [11]:
@assert sum(HMM.M.M .!= df[:,:subject]) == 0
@assert sum(HMM.X.X .!= df[:,:time]) == 0
#@assert size(HMM.Z.Z) == (400,2)

In [12]:
sigma, betaM, betaX, opt = fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

sigma = [2200.0, 14.0]
negLog = 445.681758297818
The initial object value is 445.681758297818
sigma = [2200.0, 14.0]
negLog = 445.681758297818
OPTBL: starting point [2200.0, 14.0]
sigma = [2200.0, 14.0]
negLog = 445.681758297818
sigma = [3850.0, 14.0]
negLog = 445.81040926220794
sigma = [2200.0, 24.5]
negLog = Inf
sigma = [550.0, 14.0]
negLog = 447.30774360979615
sigma = [2200.0, 3.5]
negLog = 647.6079505559085
sigma = [2216.5, 14.0]
negLog = 445.68174593672404
sigma = [2233.0, 14.0]
negLog = 445.68178871779236
sigma = [2216.664991750619, 13.895005249606285]
negLog = 445.71408729587415
sigma = [2216.5, 14.0105]
negLog = 445.6789039114469
sigma = [2216.485000687968, 14.020999566146537]
negLog = 445.6761324675332
sigma = [2218.1029034470635, 14.023060546103492]
negLog = 445.6755984940186
sigma = [2218.2572075521484, 14.023432437532758]
negLog = 445.67550230626125
sigma = [2218.19742641448, 14.024411098096314]
negLog = 445.6752490055942
sigma = [2218.298621871879, 14.025240435923726]
negL

In [38]:
sigma

2-element Vector{Float64}:
  4.824793629869606
 19.001012598276105

In [39]:
betaM

1-element Vector{Float64}:
 0.6958072278744662

In [40]:
betaX

1-element Vector{Float64}:
 1.455761621906421

## High dimensional Data

In [54]:
raw = DataFrame(CSV.File("raws.csv"))
raw = raw[1:30, 1:50]

raw[!, :response] = rand(rng, 30)
raw[!, :trt] = rand(["A","B","C"], 30)
first(raw[:,45:52],5)

,...45,...46,...47,...48,...49,...50,response,trt
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
1,0.0,0.0,0.0,0.0,0.0,0.0,0.585195,C
2,0.0,1.5,0.0,0.0,0.5,0.0,0.0773379,B
3,0.5,1.0,0.0,1.0,0.5,0.0,0.716628,B
4,0.5,1.0,0.0,0.0,0.5,0.0,0.320357,A
5,0.5,0.0,0.0,1.0,0.5,0.0,0.653093,C


In [48]:
n = size(HMM.M, 1)
A = hcat(HMM.M.M, HMM.X.X)
P = I - A*inv(transpose(A)*A)*transpose(A)
u,s,v = svd(P)
r = size(HMM.M,2) + size(HMM.X,2)  # simplify: assume fixed effect full rank
#C = randn((n-r),n)
C = transpose(u[:,1:(n-r)]) 
K = C*P
Z = HMM.Z.Z
y = HMM.y
# C can be any full rank matrix with size n,r, e.g. randn(n,r)
print()

In [49]:

function negLogLik(sigma::Vector{Float64}, g::Vector{Float64})
    n = length(y)
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    negLog = 1/2*log(det(K*Sigma*transpose(K))) + 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    #println("OPT: parameter $(sigma) || objective eval $(negLog)")
    @show sigma
    @show negLog

    return negLog
end

negLogLik (generic function with 1 method)

In [50]:
negLogLik([2200.0 , 24.5], [0.0,0.0])

sigma = [2200.0, 24.5]
negLog = Inf


Inf

In [51]:
n = length(y)

240

In [52]:
sigma = [2200.0 , 24.5]

2-element Vector{Float64}:
 2200.0
   24.5

In [53]:
Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))

240×240 Matrix{Float64}:
 2224.5  2200.0  2200.0  2200.0  2200.0  …     0.0     0.0     0.0     0.0
 2200.0  2224.5  2200.0  2200.0  2200.0        0.0     0.0     0.0     0.0
 2200.0  2200.0  2224.5  2200.0  2200.0        0.0     0.0     0.0     0.0
 2200.0  2200.0  2200.0  2224.5  2200.0        0.0     0.0     0.0     0.0
 2200.0  2200.0  2200.0  2200.0  2224.5        0.0     0.0     0.0     0.0
 2200.0  2200.0  2200.0  2200.0  2200.0  …     0.0     0.0     0.0     0.0
 2200.0  2200.0  2200.0  2200.0  2200.0        0.0     0.0     0.0     0.0
 2200.0  2200.0  2200.0  2200.0  2200.0        0.0     0.0     0.0     0.0
 2200.0  2200.0  2200.0  2200.0  2200.0        0.0     0.0     0.0     0.0
 2200.0  2200.0  2200.0  2200.0  2200.0        0.0     0.0     0.0     0.0
    0.0     0.0     0.0     0.0     0.0  …  2200.0  2200.0  2200.0  2200.0
    0.0     0.0     0.0     0.0     0.0     2200.0  2200.0  2200.0  2200.0
    0.0     0.0     0.0     0.0     0.0     2200.0  2200.0  2200.0  2200.0


In [54]:
negLog = 1/2*log(det(K*Sigma*transpose(K))) 

Inf

In [55]:
K

238×240 Matrix{Float64}:
 -2.94451e-20   0.0466504    -0.0170941   …   0.0510636   -0.00207091
 -2.05322e-19  -0.0496783     0.0395708      -0.0363789    0.010993
  6.41966e-19   0.000171218   0.0519133      -0.0049346   -0.00377829
 -1.34922e-18   0.131597     -0.075034        0.0351687   -0.0151265
 -3.5239e-19   -0.0956999    -0.00164722     -0.0416952    0.00499927
  6.46778e-19  -0.0898978    -0.0466286   …   0.00544828   0.00859205
 -3.39007e-19   0.098675     -0.0658652      -0.00823248  -0.00756623
 -8.48762e-20  -0.0308788     0.0415333      -0.0100817    0.00404112
 -3.37464e-19  -0.0194747     0.0635107       0.0134814    0.00548691
  2.83801e-19   0.00314105   -0.0375957      -0.0462907    0.000495605
  3.07301e-19  -0.0733903    -0.0431255   …   0.0453955    0.00810663
  7.40189e-19   0.0467832     0.0896087       0.0377275    0.00471873
 -6.13821e-19  -0.0797151    -0.0309555      -0.00292084  -0.00690537
  ⋮                                       ⋱               
  5.7218

In [56]:
det(K*Sigma*transpose(K))

Inf

In [57]:
K*Sigma*transpose(K)

238×238 Matrix{Float64}:
   719.818    -885.513      -893.559   …   -527.868     638.14    -1036.08
  -885.513    1159.67       1102.21         685.224    -803.221    1262.6
  -893.559    1102.21       1344.99         615.968    -865.673    1605.26
   -55.5432     83.1054       11.8485        63.742     -35.2111    -11.8998
  1027.75    -1344.58      -1148.96        -842.515     897.737   -1258.21
 -1044.72     1368.28       1160.66    …    859.05     -910.632    1267.44
 -1513.39     1920.96       1975.68        1137.77    -1397.1      2304.06
  -303.088     414.907       250.346        280.529    -241.314     230.345
  -634.736     791.179       898.396        451.907    -604.438    1077.3
   729.222    -925.961      -950.268       -548.846     672.735   -1107.49
  1372.77    -1793.72      -1545.41    …  -1121.45     1201.95    -1697.69
 -1190.64     1476.36       1722.43         834.2     -1143.66     2079.99
    55.0928    -82.7592      -10.1754       -63.7967     34.508      14.31

In [58]:
rank(K*Sigma*transpose(K))

238